In [ ]:
# !pip install langchain-chroma

In [87]:
import os

os.environ["HF_HOME"] = "/space/hotel/phit/personal/experiments/weights"
os.environ["TORCH_HOME"] = "/space/hotel/phit/personal/experiments/weights"

from typing import List, Optional, Union

from langchain.callbacks import FileCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, JSONLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS , Chroma
# from langchain_chroma import Chroma
from langchain_core.documents import Document
from loguru import logger
from rich import print
from sentence_transformers import CrossEncoder

from unstructured.cleaners.core import clean_extra_whitespace, group_broken_paragraphs

logfile = "log/output.log"
logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)


persist_directory = None




class RAGException(Exception):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


def rerank_docs(reranker_model, query, retrieved_docs):
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)


def load_pdf(
    files: Union[str, List[str]] = "../data/cv/Bui Tien Phat resume (1).pdf"
) -> List[Document]:
    if isinstance(files, str):
        loader = UnstructuredFileLoader(
            files,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        return loader.load()

    loaders = [
        UnstructuredFileLoader(
            file,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs

VIETNAMWORKS = ['_id', 'url', 'job_name', 'company_name', 'salary', 'end_date',
       'address', 'posted_date', 'job_function', 'job_industry', 'job_level',
       'skill', 'preferred_language', 'job_description'] # job_requirements

TOPCV = ['_id', 'urls', 'job_name', 'company_name', 'address', 'salary',
       'remaining', 'job_description', 'benefits',
       'application_method', 'level', 'experience', 'number_of_recruitment',
       'work_form', 'gender', 'working_time'] # job_requirements

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    
    # metadata["urls"] = record.get("urls")
    # metadata["job_name"] = record.get("job_name")
    # metadata["company_name"] = record.get("company_name")
    # metadata["address"] = record.get("address")
    # metadata["salary"] = record.get("salary")
    # metadata["remaining"] = record.get("remaining")
    # # metadata[""] = record.get("Mô tả công việc")
    # metadata["Yêu cầu ứng viên"] = record.get("Yêu cầu ứng viên")
    # metadata["Quyền lợi"] = record.get("Quyền lợi")
    # metadata["Địa điểm làm việc"] = record.get("Địa điểm làm việc")
    # metadata["Cách thức ứng tuyển"] = record.get("Cách thức ứng tuyển")
    # metadata["Cấp bậc"] = record.get("Cấp bậc")
    # metadata["Kinh nghiệm"] = record.get("Kinh nghiệm")
    # metadata["Số lượng tuyển"] = record.get("Số lượng tuyển")
    # metadata["Hình thức làm việc"] = record.get("Hình thức làm việc")
    # metadata["Giới tính"] = record.get("Giới tính")
    
    for key in VIETNAMWORKS:
        metadata[key] = record.get(key)
    
    metadata = {key: f'{" ".join(val[0]) if isinstance(val, list) else val}' for key, val in metadata.items()}

    

    return metadata


def load_jsonl(
    files: Union[str, List[str]] = "../data/crawl/train_test.jsonl"
) -> List[Document]:
    if isinstance(files, str):
        loader = JSONLoader(
            files,
            json_lines=True,
            jq_schema='.',
            content_key="job_requirements", 
            text_content=False,
            metadata_func=metadata_func
        )
        return loader.load()

    loaders = [
        JSONLoader(
            file,
            json_lines=True,
            jq_schema='.messages[]',
            content_key="content",
            metadata_func=metadata_func
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs

def create_parent_retriever(
    docs: List[Document], embeddings_model: HuggingFaceBgeEmbeddings()
):
    parent_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=2000,
        length_function=len,
        is_separator_regex=False,
    )

    # This text splitter is used to create the child documents
    child_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=1000,
        chunk_overlap=300,
        length_function=len,
        is_separator_regex=False,
    )
    # The vectorstore to use to index the child chunks
    vectorstore = Chroma(
        collection_name="split_documents",
        embedding_function=embeddings_model,
        persist_directory=persist_directory,
    )#.as_retriever()
    
    # vectorstore = Chroma.from_documents(docs, embeddings_model)#.as_retriever()
    print("vectorstore: ", vectorstore)
    # The storage layer for the parent documents
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        k=10,
    )
    retriever.add_documents(docs)
    return retriever


def retrieve_context(query, retriever, reranker_model):
    retrieved_docs = retriever.get_relevant_documents(query)

    if len(retrieved_docs) == 0:
        raise RAGException(
            f"Couldn't retrieve any relevant document with the query `{query}`. Try modifying your question!"
        )
    reranked_docs = rerank_docs(
        query=query, retrieved_docs=retrieved_docs, reranker_model=reranker_model
    )
    return reranked_docs

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3
def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5", device: str = "cuda"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/llm_reranker
def load_reranker_model(
    reranker_model_name: str = "BAAI/bge-reranker-large", device: str = "cuda"
) -> CrossEncoder:
    reranker_model = CrossEncoder(
        model_name=reranker_model_name, max_length=512, device=device
    )
    return reranker_model


def main(
    file: str = "../data/crawl/train_test.jsonl",
    query: Optional[str] = None,
    llm_name="mistral",
):
    # docs = load_pdf(files=file)
    docs = load_jsonl("../data/crawl/vnw.jsonl")
    # print(docs)

    embedding_model = load_embedding_model()
    retriever = create_parent_retriever(docs, embedding_model)
    reranker_model = load_reranker_model()

    context = retrieve_context(
        query, retriever=retriever, reranker_model=reranker_model
    )[0]
    print("context:\n", context, "\n", "=" * 50, "\n")


# if __name__ == "__main__":
#     # from jsonargparse import CLI

#     # CLI(main)
#     # main(query="What is the job description for Network Engineer?")
#     main(query="What is job description in Da Nang ?")
    

In [89]:
docs = load_jsonl('../data/preprocessed/preprocessed_vnw.jsonl')
# print(docs)

embedding_model = load_embedding_model(model_name="BAAI/bge-m3")
retriever = create_parent_retriever(docs, embedding_model)
reranker_model = load_reranker_model(reranker_model_name="BAAI/bge-reranker-v2-m3")

vectorstore:  <langchain_community.vectorstores.chroma.Chroma object at 0x7f19a0648f40>

In [90]:
import os
files = ["../data/cv/" + file for file in os.listdir("../data/cv")]
files

['../data/cv/CV Cap Tan Dat - Intern Developer.pdf',
 '../data/cv/CV_SonBao_DS.pdf',
 '../data/cv/Bui Tien Phat resume (1).pdf',
 '../data/cv/CV Nguyễn việt hoàng - CV lập trình-TopCV.vn.pdf',
 '../data/cv/CV _ Đinh Tuấn Nam_BE Developer(.NET)_HCM.pdf',
 '../data/cv/CV_MaiHuyKhang_DA.pdf',
 '../data/cv/CV_NGUYEN_MINH_SON.pdf']

In [ ]:
cv = load_pdf(files)

In [97]:
print(cv[2].page_content)

Data Science/AI Engineer

TIEN PHAT BUI

0962980173 | buitienphat2462002@gmail.com https://www.linkedin.com/in/tien-phat-bui-ba3421222/ 
https://github.com/ShynBui/ Go Vap district, Ho Chi Minh city

Education

B ACH EL OR OF IN FOR M ATION TECH N OL O G Y(Expected 05/2024) – Ho Chi Minh City Open University – 97 Vo Van Tan,
Ward 6, District 3, Ho Chi Minh City GPA: 3.76/4.00 (Highest in Major) Majors: Information Technology Relevant 
Coursework: Machine Learning, Data Visualization and Communication, Database Systems and SQL, Natural Language 
Processing, Computer Vision, Deep Learning and Neural Networks

Skills

Python Libraries and frameworks: pandas, matplotlib, numpy, BeutifulSoup4, re, os, scikit-learn, Hugging Face, 
transformers, PyTorch, Python-Flask, LangChain, NLTK Big Data Apache Hadoop, PySpark SQL Software: SQLite, MySQL, 
SQL, PostgreSQL Power Bi Tools: Power Query, Visualize C++ Libraries: STL, Boot C++, Armadillo

Projects

LEGAL SEAR CH AN D R ES PON SE PLATFOR M – Group competition project

November 2023 

-

 now

● Provides users with quick access to legal information, delivers reliable answers to legal queries, and enables 
efficient navigation through legal codes and documents

● Utilizes ReactJS for front

-end development, employs Python Flask for back

-end functionality, and enhances its capabilities with complementary integration tools such as CI/CD (Continuous 
Integration/Continuous Deployment) and Docker for streamlined deployment processes, ensuring efficient and reliable
operation

● Project management, ensuring a waterfall system development process, data search and preprocessing, training deep
learning models based on PhoBERT and LangChain for question

-answering tasks for answering legal and training another model to classify "legal definition" sentences in legal 
documents, taking on the role of data designer using MySQL, and visualize the model training process for reporting 
using matplotlib

Winning third prize in the Vietnam Student Informatics Olympiad - Open source competition, the project attracted 
the jury for its creativity and was considered to have the most potential. The product is open source code that is 
being invested in, developed and raised capital The article introduces the project: 
https://vfossa.vn/tin-tuc/code-heroes-nen-tang-tra-cuu-va-giai-dap-phap-luat-ung-dung-sang- 
tao-cac-mo-hinh-ngon-ngu-lon-698.html

AN AUTOM ATICALLY AN S WER IN G EN GLISH AN D VIETN AM ESE R EA D IN G COM PR EH EN SION S YSTEM – Personal June 
2021 - January 2022 Project

● Assisting users in automatically answering Vietnamese or English reading comprehension questions by providing the
system with

a passage of information (provided by the user), from which the system extracts information to answer the user's 
queries ● Using Hugging Face as a model repository, we employ a BERT-based model for answering English reading 
comprehension

Using Hugging Face as a model repository, we employ a BERT-based model for answering English reading comprehension 
questions and a PhoBERT-based model for answering Vietnamese reading comprehension questions. Gradio is utilized to
design an interactive interface for user interaction

questions and a PhoBERT-based model for answering Vietnamese reading comprehension questions. Gradio is utilized to
design an interactive interface for user interaction Taking on the role of data preparation, I utilize available 
datasets and scrape data from the web using BeautifulSoup4. Data processing is performed using Python, while model 
fine-tuning is carried out through the use of Transformers and PyTorch. Model evaluation and visualization of 
evaluation results are conducted using matplotlib The English question-answering model achieved a significant 78% 
accuracy when applied to multiple-choice questions. The Vietnamese question-answering model also achieved an exact 
match accuracy of 72%





OTH ER PR OJECT

Online library ● ● Website for book sales management ● 

In [98]:
# query = "What are some job descriptions for Data?"
query = cv[2].page_content
context = retrieve_context(
    query, retriever=retriever, reranker_model=reranker_model
)[:]
print("context:\n", context, "\n", "=" * 50, "\n")

context:

[
    (
        Document(
            page_content='"- Bachelors degree in Computer Science, Electrical Engineering, or a related field or 3+
years of equivalent experience.", - At least 4 + years of experience in AI or relevant positions., - Have 2 + year 
of team leader experience or relevant positions., - Knowledge of the following areas (but not limited to): 
Recommendation system, - Computer Vision, NLP (information extraction), Large Language Models, OCR pipeline., - 
Knowledge of AI frameworks: Pytorch, Tensorflow, Triton, TF serving., - Experience in writing API to wrap AI 
models., - Ability to plan & organize AI services, especially in emergency situations., - Excellent problem-solving
and analytical skills., - Be proactive, enthusiastic, hard-working., - Be integrity, careful, accurate., *** What 
we offer, - Working time from Monday to Friday (09:00 am - 18:00 pm)., - Good chance to study and develop career 
path stably with internal promotion and rotation., - 100% statutory insurance contribution as per the labor law., -
Bao Viet Healthcare insurance., - 100% gross salary in probation., - 12 Annual Leave (AL) per year + extra 1 AL per
extra working year., - Annual bonus: depending on business & individual performance., - Annual salary review., - 
ESOP allows you to financially share in our future success., - Other rewards: 5-star, employee of the month (EOTM),
employee of the year (EOTY), loyalty employee (3-year, 5-year, 8-year, etc.), "- Internal Activities: General 
Meeting, Company Trip, International Womens Day,", "Vietnamese Womens Day Mid-Autumn Festival, Family Day, 
Christmas, Year-end Party, etc."',
            metadata={
                'source': 
'/mnt/net/i2x256-ai03/hotel/phit/personal/applied-data-science/data/preprocessed/preprocessed_vnw.jsonl',
                'seq_num': '397',
                '_id': '661621edbb86ac211a66ec97',
                'url': 
'https://www.vietnamworks.com/ai-manager--1759990-jv?source=searchResults&searchType=2&placement=1759990&sortBy=lat
est',
                'job_name': 'AI Manager',
                'company_name': 'Got It',
                'salary': 'Thương lượng',
                'end_date': '2024-04-28',
                'address': '9-11 Nguyễn Văn Thủ, P. Đa Kao, Quận 1, TP. Hồ Chí Minh',
                'posted_date': '29/03/2024',
                'job_function': 'Công Nghệ Thông Tin/Viễn Thông,  > , Phần Mềm Máy Tính',
                'job_industry': 'Phần Mềm CNTT/Dịch vụ Phần mềm',
                'job_level': 'Trưởng phòng',
                'skill': 'TensorFlow, Computer Science, Application Development, Product Development, Pipeline 
Development',
                'preferred_language': 'Bất kỳ',
                'job_description': 'What you need to do:, - Lead and deliver AI solutions for Dayone, including 
(but not limited to): maximizing Got It voucher’s revenue (e.g. intelligent voucher redemption recommendation 
system), online shopping recommendation systems, AI-assisted productivity boosting applications, and OCR system for
invoices and business documents., - Work with large and real world data from our products (Got It, Buzz It, Scan 
It, and internal tools) to generate insightful information, assist business decisions, and maximize business value 
of our products., - Develop tools and frameworks to help training, deploying, and monitoring AI models more generic
and effective., - Organize AI team to optimize resources yet deliver high quality services., - Collaborate with 
other teams to assure the service operations meet expectations from internal and external users., - Perform 
additional duties as required by Line Manager.'
            }
        ),
        0.72114486
    )
]

 ==================================================